
# GPU Numba and CuPy Parallelization of Matrix Multiplication 

Similary to the multicore parallelization lab, in this lab we will be using Numba and CuPy to accelerate matrix-matrix multiplications using GPU. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba for CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html)
* [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb)
* [Numba.CUDA by Graham Markell](https://github.com/numba/nvidia-cuda-tutorial)
* [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/)
* [CuPy Basics](https://docs.cupy.dev/en/stable/user_guide/basic.html)

[//]: <> (GEOPHYS 257 Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

<br>

### Exercise 0

* You need to request a *T4* node on the cluster. Don't forget that you need to add **--gres=gpu** to your srun command.
* Reminder: on the *T4* nodes you need to load a different version of Python:
```bash
spack load python@3.10.7
```

* Import every Python module, object, and/or function that you need below.

In [ ]:
# load env
# srun -p T4 --gres=gpu:1 --pty bash --x11
srun --partition=t4 --gres=gpu:1 --x11 --exclusive --pty /bin/bash

. /home/spack/spack/share/spack/setup-env.sh
spack load gcc@9.5.0
spack compiler find
spack load python@3.10.7

<br>

### Exercise 1: Matrix Transpose

Before we examine matrix-matrix multiplication, we will first write a GPU kernel that transposes a square matrix.  This type of problem is a good introduction into how to use the CUDA threading model. The task for this exercise is to write a Numba CUDA kernel that will transpose a square matrix. 

**Before you start**, take a look at the following:
* Read over the following notebook that explanes Numba.CUDA kernels: [Writing Numba.CUDA kernels Notebook](https://github.com/ContinuumIO/gtc2017-numba/blob/master/4%20-%20Writing%20CUDA%20Kernels.ipynb) 
* The first matrix-matrix multiplication code (the one that **doesn't** use shared memory) shown at [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). Understanding this code should give a pretty good idea on how to write the transpose kernel. The matrix-matrix kernel code from the NYU lab is shown below.
```python
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
```

**Tasks for this exercise**
* Write a Numba.CUDA kernel that transpose an $NxN$ square matrix.
* Be sure that the transpose kernel can transpose square matrices with sizes of $N$ as small as $N=2$ and as large as $N=10240$.
* Using shared memory is **not** required.


In [1]:
import numpy as np
from numba import cuda, jit

# CUDA kernel
@cuda.jit
def mattran(A, B):
    """ Perform the matrix transpose of a NxN square matrix, i.e., A = B^T
    """
    
    # obatin the index
    xpos, ypos = cuda.grid(2)
    
    # perform the transpose
    if xpos < A.shape[0] and ypos < A.shape[1]:
        B[ypos, xpos] = A[xpos, ypos] 


# Host code

# Initialize the data arrays
N = 1024
A = np.linspace(0, N*N,num=N*N, endpoint=True).reshape(N, N)
d_A = cuda.to_device(A)
d_B = cuda.device_array((N, N))

# Configure the blocks
threadsperblock = (16, 16)
blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(A.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

# Start the kernel 
mattran[blockspergrid, threadsperblock](d_A, d_B)

# Copy the result back to the host
B = d_B.copy_to_host()

In [2]:
print(A)
print('\n')
print(B)
assert np.any(A == B.T)

[[0.00000000e+00 1.00000095e+00 2.00000191e+00 ... 1.02100097e+03
  1.02200097e+03 1.02300098e+03]
 [1.02400098e+03 1.02500098e+03 1.02600098e+03 ... 2.04500195e+03
  2.04600195e+03 2.04700195e+03]
 [2.04800195e+03 2.04900195e+03 2.05000196e+03 ... 3.06900293e+03
  3.07000293e+03 3.07100293e+03]
 ...
 [1.04550500e+06 1.04550600e+06 1.04550700e+06 ... 1.04652600e+06
  1.04652700e+06 1.04652800e+06]
 [1.04652900e+06 1.04653000e+06 1.04653100e+06 ... 1.04755000e+06
  1.04755100e+06 1.04755200e+06]
 [1.04755300e+06 1.04755400e+06 1.04755500e+06 ... 1.04857400e+06
  1.04857500e+06 1.04857600e+06]]


[[0.00000000e+00 1.02400098e+03 2.04800195e+03 ... 1.04550500e+06
  1.04652900e+06 1.04755300e+06]
 [1.00000095e+00 1.02500098e+03 2.04900195e+03 ... 1.04550600e+06
  1.04653000e+06 1.04755400e+06]
 [2.00000191e+00 1.02600098e+03 2.05000196e+03 ... 1.04550700e+06
  1.04653100e+06 1.04755500e+06]
 ...
 [1.02100097e+03 2.04500195e+03 3.06900293e+03 ... 1.04652600e+06
  1.04755000e+06 1.04857400e+0

<br>

### Exercise 2: Using Numba CUDA to parallelize matrix multiplication: 

For this exercise, we will use Numba compiled GPU kernels that calculate matrix-matrix multiplication for square matrices. In particular, we will use a GPU kernel that doesn't used shared memory and compared to a GPU kernel that does use shared-memory. Please use the two kernel codes discussed in the following lab: [NYU Numba CUDA Lab5](https://nyu-cds.github.io/python-numba/05-cuda/). As you will see in this exercise, learning to use shared-memory (akin to user-controlled cache), can take a lot of practice, so in the next exercise, we examine how well the simple shared-memory kernel from the NYU lab compares to the optimized codes provided by NVIDIA in the CuPy package. 

#### The tasks for this exercise:
1. Copy the matrix-matrix kernel codes from the NYU lab. Test them for accurracy against *numpy.dot()* and also compare time runtimes these GPU kernels the numpy.dot() function as well. **Note:** Use [CUDA events](https://numba.readthedocs.io/en/stable/cuda-reference/host.html#events) when timing GPU kernel calls because the driver does not "block" the calling process (for case this is IPython). Insted, the kernel is sent to the GPU to run, and then the process (IPython) immediately continues to it's next bit of code. Contrary to GPU kernel calls, calls to copy data to or from the GPU will block the process. For these cases, the calls can be timed the same way that other Python calls are timed.<br> **For both GPU kernels:**
    - Test with square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 5120$, $N=10240$, and $N=20480$. **Tip**, first make sure you can get the GPU codes to work and that you get correct results by testing with $N_{test}=32$.
    - For each $N$ above, test the multiplication for both dtypes: *dtype=float32* and *dtype=float64*.
    - Calculate and show the error between your functions and the *numpy.dot()* function. 
    - Calculate and show the *speedup* (or *slowdown*) of your GPU kernel for each $N$ vs *numpy.dot()*. Be sure to include the array copy times in the "total-gpu-kernel runtime.
    - For each $N$ vs, calculate and show the *speedup* of your GPU kernel using *dtype=float32* vs *dtype=float64*. Be sure to include the array copy times in the "total-gpu-kernel runtime."
    
<br>

2. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
h_A = np.random.random((N, N)).astype(np.<float-type>)
h_B = np.random.random((N, N)).astype(np.<float-type>)
```    
<br>

3. For the device memory:
    - Create **d_A** and **d_B** by copying **h_A** and **h_B** to the GPU, and be sure to time the copies
    - Create **d_C** as device-array that is allocated on the GPU (device) only, and not on the host (**Do Not Copy**)
    
<br>

4. After the GPU matrix-matrix multiplication kernel finishes, **copy** the the *device-array* **d_C** to the *host-array* **h_C**, and be sure to time this copy.

<br>

5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the speedup or slowdowns vs numpy as well as float32 vs float64. Remember, that your runtime for the GPU kernel include time to compile the kernel (not much you can do to control this). Futhermore, becasue you have to copy data to and off of the GPU, these copy times should be included in the "total-gpu-kernel runtime." 

In [2]:
import numpy as np
from numba import cuda, float32, float64
from timeit import default_timer as timer
from __future__ import division


# CUDA kernel
@cuda.jit
def matmul(A, B, C):
    """Perform matrix multiplication of C = A * B
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp
        

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul_float32(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp
    
    
@cuda.jit
def fast_matmul_float64(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float64)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float64)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    
    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [11]:
# Tests

for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:
        
        # Initialize the data arrays
        h_A = np.random.random((N, N)).astype(dtype)
        h_B = np.random.random((N, N)).astype(dtype)
        
        # Create cuda events
        evt_beg = cuda.event()
        evt_end = cuda.event()

        # Begin recording the time 
        evt_beg.record()

        # Copy the arrays to the device
        d_A = cuda.to_device(h_A)
        d_B = cuda.to_device(h_B)
        d_C = cuda.device_array((N, N))

        # Configure the blocks
        threadsperblock = (16, 16)
        blockspergrid_x = int(np.ceil(h_A.shape[0] / threadsperblock[0]))
        blockspergrid_y = int(np.ceil(h_B.shape[1] / threadsperblock[1]))
        blockspergrid = (blockspergrid_x, blockspergrid_y)

        # Start the kernel 
        matmul[blockspergrid, threadsperblock](d_A, d_B, d_C)

        # Copy the result back to the host
        h_C = d_C.copy_to_host()

        # End recording the time 
        evt_end.record()
        evt_end.synchronize()

        # Perform Numpy solution 
        np_beg = timer()
        np_C = np.dot(h_A, h_B)
        np_end = timer()

        # Print information
        time_gpu = evt_beg.elapsed_time(evt_end)
        time_np = float(np_end - np_beg) * 1000

        print('Case: N = {}, dtype = {}:'.format(N, dtype))
        print('  GPU   time: {:f} ms'.format(time_gpu))
        print('  Numpy time: {:f} ms'.format(time_np))
        print('  Speed up  : {:f}'.format(time_np/time_gpu))
        print('  Rela error: {:.6e}'.format(np.sum(abs(np_C - h_C))/np.sum(abs(np_C))))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  GPU   time: 1104.032959 ms
  Numpy time: 240.688116 ms
  Speed up  : 0.218008
  Rela error: 8.205821e-08


Case: N = 5120, dtype = <class 'numpy.float64'>:
  GPU   time: 1694.944580 ms
  Numpy time: 389.132014 ms
  Speed up  : 0.229584
  Rela error: 1.650166e-15


Case: N = 10240, dtype = <class 'numpy.float32'>:
  GPU   time: 5840.136230 ms
  Numpy time: 1404.815898 ms
  Speed up  : 0.240545
  Rela error: 8.061929e-08


Case: N = 10240, dtype = <class 'numpy.float64'>:
  GPU   time: 11165.965820 ms
  Numpy time: 2539.452701 ms
  Speed up  : 0.227428
  Rela error: 2.326184e-15


Case: N = 20480, dtype = <class 'numpy.float32'>:
  GPU   time: 44746.722656 ms
  Numpy time: 8447.079863 ms
  Speed up  : 0.188775
  Rela error: 9.866385e-08


Case: N = 20480, dtype = <class 'numpy.float64'>:
  GPU   time: 86922.656250 ms
  Numpy time: 21779.138505 ms
  Speed up  : 0.250558
  Rela error: 3.287642e-15




**Response**: when N = 5120, the numpy.dot() function is faster than the GPU kernel for both the np.float32 and np.float64 data types. But when N increases to 10240 and 20480, the GPU kernel is faster than the numpy.dot() function. And the speedup is more significant when N is becoming large.

In [12]:
# Tests

for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:
        
        # Initialize the data arrays
        h_A = np.random.random((N, N)).astype(dtype)
        h_B = np.random.random((N, N)).astype(dtype)
        
        # Create cuda events
        evt_beg = cuda.event()
        evt_end = cuda.event()

        # Begin recording the time 
        evt_beg.record()

        # Copy arrays to the device
        d_A = cuda.to_device(h_A)
        d_B = cuda.to_device(h_B)
        d_C = cuda.device_array((N, N), dtype=dtype)

        # Configure the blocks
        threadsperblock = (TPB, TPB)
        blockspergrid_x = int(np.ceil(h_A.shape[0] / threadsperblock[1]))
        blockspergrid_y = int(np.ceil(h_B.shape[1] / threadsperblock[0]))
        blockspergrid = (blockspergrid_x, blockspergrid_y)

        # Start the kernel based on data type
        if dtype is np.float32:
            fast_matmul_float32[blockspergrid, threadsperblock](d_A, d_B, d_C)
        elif  dtype is np.float64:
            fast_matmul_float64[blockspergrid, threadsperblock](d_A, d_B, d_C)
        else:
            raise ValueError(f'Not support dtype {dtype}')
            
        # Copy the result back to the host
        h_C = d_C.copy_to_host()

        # End recording the time 
        evt_end.record()
        evt_end.synchronize()

        # Perform Numpy solution 
        np_beg = timer()
        np_C = np.dot(h_A, h_B)
        np_end = timer()

        # Print information
        time_gpu = evt_beg.elapsed_time(evt_end)
        time_np = float(np_end - np_beg) * 1000

        print('Case: N = {}, dtype = {}:'.format(N, dtype, ))
        print('  GPU   time: {:f} ms'.format(time_gpu))
        print('  Numpy time: {:f} ms'.format(time_np))
        print('  Speed up  : {:f}'.format(time_np/time_gpu))
        print('  Rela error: {:.6e}'.format(np.sum(abs(np_C - h_C))/np.sum(abs(np_C))))
        print('\n')

  GPU   time: 948.466492 ms
  Numpy time: 214.667926 ms
  Speed up  : 0.226332
  Rela error: 7.901321e-08


  GPU   time: 2065.178223 ms
  Numpy time: 359.335531 ms
  Speed up  : 0.173997
  Rela error: 1.650352e-15


  GPU   time: 4056.052002 ms
  Numpy time: 1216.757579 ms
  Speed up  : 0.299986
  Rela error: 7.756453e-08


  GPU   time: 13147.020508 ms
  Numpy time: 2591.687422 ms
  Speed up  : 0.197131
  Rela error: 2.326432e-15


  GPU   time: 30855.353516 ms
  Numpy time: 10527.211300 ms
  Speed up  : 0.341179
  Rela error: 9.618172e-08


  GPU   time: 102773.500000 ms
  Numpy time: 21981.716861 ms
  Speed up  : 0.213885
  Rela error: 3.287668e-15




**Response**: when using the shared memory, the GPU kernel is faster than the numpy.dot() function in all cases, including N = 5120, 10240 and 20480, and for both date type of float32 and float64. Also, the speedup is more significant when N is becoming large. 

<br>

### Exercise 3: CuPy 

For this exercise, we will repeat what we did in *Exercise 2*. However, we will use *CuPy* functions, which are similar to *Numpy* funcstions with some added functions for copying data to-the-device-from-the-host and to-the-host-from-the-device. By using CuPy, we can depend on code that has been optimized for the GPU by NVIDIA, and instead of tyring to optimize our matrix-matrix multiplication kernels, we can use a built-in function to calculate the multiplication instead (i.e. [cupy.dot()](https://docs.cupy.dev/en/stable/reference/generated/cupy.dot.html#cupy.dot)).

**Tasks for this exercise:**
* Same as those listed in *Exercise 2*, but compare *cupy.dot()* to *numpy.dot()*.
* Also, reuse the host-arrays, *h_A* and *h_B* above. You will need to call the appropriate *CuPy* fuctions to copy these arrays to the GPU and to copy the result back to the host. You will **not** need to declare the deive-C array before calling *cupy.dot()* because the function will do it for you (like numpy does).


In [4]:
# Tests
import cupy as cp

for N in [5120, 10240, 20480]:
    for dtype in [np.float32, np.float64]:
        
        # Initialize the data arrays
        h_A = cp.random.randn(N, N)
        h_B = cp.random.randn(N, N)
        
        # Create cuda events
        evt_beg = cuda.event()
        evt_end = cuda.event()

        # Begin recording the time 
        evt_beg.record()

        # Multiply the two matrices using Cupy's dot() function
        cp_C = cp.dot(h_A, h_B)

        # End recording the time 
        evt_end.record()
        evt_end.synchronize()

        # Perform Numpy solution 
        np_beg = timer()
        np_C = np.dot(h_A, h_B)
        np_end = timer()

        # Print information
        time_gpu = evt_beg.elapsed_time(evt_end)
        time_np = float(np_end - np_beg) * 1000

        print('Case: N = {}, dtype = {}:'.format(N, dtype, ))
        print('  GPU   time: {:f} ms'.format(time_gpu))
        print('  Numpy time: {:f} ms'.format(time_np))
        print('  Speed up  : {:f}'.format(time_np/time_gpu))
        print('  Rela error: {:.6e}'.format(np.sum(abs(np_C - cp_C))/np.sum(abs(np_C))))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  GPU   time: 669.494263 ms
  Numpy time: 3.706200 ms
  Speed up  : 0.005536
  Rela error: 0.000000e+00


Case: N = 5120, dtype = <class 'numpy.float64'>:
  GPU   time: 42.227711 ms
  Numpy time: 0.062447 ms
  Speed up  : 0.001479
  Rela error: 0.000000e+00


Case: N = 10240, dtype = <class 'numpy.float32'>:
  GPU   time: 320.380920 ms
  Numpy time: 1.013658 ms
  Speed up  : 0.003164
  Rela error: 0.000000e+00


Case: N = 10240, dtype = <class 'numpy.float64'>:
  GPU   time: 320.115723 ms
  Numpy time: 0.106688 ms
  Speed up  : 0.000333
  Rela error: 0.000000e+00


Case: N = 20480, dtype = <class 'numpy.float32'>:
  GPU   time: 2596.072510 ms
  Numpy time: 187.907822 ms
  Speed up  : 0.072382


OutOfMemoryError: Out of memory allocating 3,355,443,200 bytes (allocated so far: 14,260,633,600 bytes).

In [5]:
!nvidia-smi

Fri Feb 24 11:12:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    36W / 250W |  13323MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:5E:00.0 Off |                    0 |
| N/A   

<br>

### Exercise $\mathbf{\pi}$: CuPy Interoperability

Numba and CuPy device arrays (GPU arrays) can be accept each other's arrays. See [Interoperability](https://docs.cupy.dev/en/stable/user_guide/interoperability.html).

**Tasks for this exercise**
* Use the **device** arrays, **d_A** and **d_B**, that were created in *Exercise 2* to calculate the matrix-matrix multiplcation using *cupy.dot()*.
* Verify that you get the same results as you did in *Exercise 3*.
* You will need to "wrap" the device arrays before passing them to *cupy.dot()*. Read the *Interoperability* documentation linked above.
    - Time how long it takes (runtime) to "wrap" these arrays.
    - Compare this runtime to the runtime it took to create the device arrays in *Exercise 3*.
    - Provide a quick comment your thoughts on the runtime differences compared above.

In [10]:
# Tests
import numpy as np
from numba import cuda, float32, float64
import cupy as cp
from timeit import default_timer as timer

for N in [5120]: # , 10240, 20480
    for dtype in [np.float32]: # , np.float64
        
        # Initialize the data arrays
        h_A = np.random.random((N, N)).astype(dtype)
        h_B = np.random.random((N, N)).astype(dtype)
        
        # Copy the arrays to the device
        cuda_beg = timer()
        d_A = cuda.to_device(h_A)
        d_B = cuda.to_device(h_B)
        d_C = cuda.device_array((N, N))
        cuda_end = timer()

        
        cupy_beg =  timer()
        cp_A = cp.asarray(d_A)
        cp_B = cp.asarray(d_B)
        cupy_end = timer()

        cp_C = cp.dot(cp_A, cp_B)

        # Perform Numpy solution 
        np_beg = timer()
        np_C = np.dot(h_A, h_B)
        np_end = timer()

        # Print information
        time_np   = float(np_beg - np_end) * 1000
        time_cuda = float(cuda_end - cuda_beg) * 1000
        time_cupy = float(cupy_end - cupy_beg) * 1000

        print('Case: N = {}, dtype = {}:'.format(N, dtype, ))
        print('  Cuda time: {:f} ms'.format(time_cuda))
        print('  Cupy time: {:f} ms'.format(time_cupy))
        # print('  Rela error: {:.6e}'.format(np.sum(abs(np_C - cp_C))/np.sum(abs(np_C))))
        print('\n')

Case: N = 5120, dtype = <class 'numpy.float32'>:
  Cuda time: 45.512715 ms
  Cupy time: 3.446711 ms


